In [1]:
import requests

In [2]:
import bs4 as bs

In [3]:
from bs4 import BeautifulSoup

In [4]:
import numpy as np

In [5]:
import json

In [6]:
import pandas as pd

In [7]:
import re

In [8]:
##Open list of run ids
with open('/users/ssunilkumar/desktop/cos_data/etcrunid_list.txt') as textfile:
    contents = textfile.readlines()
run_ids = sorted([x.strip() for x in contents])

In [9]:
##Create empty list to store run ids
results_list = []

In [10]:
##Populate URLs to retrieve results pages
for EtcRunId in run_ids:
    URL = 'https://etc.stsci.edu/etc/results/'+str(EtcRunId)
    results_list.append(URL)

In [11]:
##Generate empty list of URLs
objnumbers = []

In [12]:
##Generate list to store version numbers in
etc_version = []

In [13]:
##Generate list to store calculated exposure times in
exposure_times = []

In [14]:
##Generate list to store calculated S/N in
signal_to_noise = []

In [15]:
##Generate list to store total count rates in
count_rate = []

In [16]:
##Generate list to store type of input spectrum in
input_spectrum_type = []

In [17]:
##Generate list to store chosen background in
background_chosen = []

In [18]:
##List of bad URLs
modified_range = []

In [ ]:
##Gather count rates
for filenumber in results_list:
    ##Retrieve URLs
    objnumber = requests.get(filenumber)
    ##Parse with html parser
    soup = bs.BeautifulSoup(objnumber.text, 'html.parser')
    ##Find 'plain' class
    block = soup.find_all(class_ = 'plain')
    for element in block:
        ##Append total count rate to list
        if 'Total in selected region' in element.get_text():
            formatted_count_rate = (str(element)).split('\n')[2]
            numerical_rate = formatted_count_rate.split('>')[1].split('<')[0]
            if ',' in numerical_rate:
                float_rate = float(numerical_rate.replace(',', ''))
            else:
                float_rate = float(numerical_rate)
            count_rate.append(float_rate)

In [ ]:
##Gather S/N and Exp time estimates
for filenum in results_list:
    objnumber = requests.get(filenum)
    soup = bs.BeautifulSoup(objnumber.text, 'html.parser')
    block = soup.find_all(class_ = 'primary')
    for element in block:
        if 'Signal/Noise Ratio' in element.get_text():
            formatted_sn = float(str(element).split('= ')[1].split('\n')[0])
            signal_to_noise.append(formatted_sn)
        elif 'Time' in element.get_text():
            formatted_exptime = str(element).split('= ')[1].split(' ')[0]
            if ',' in formatted_exptime:
                float_exptime = float(formatted_exptime.replace(',', ''))
            else:
                float_exptime = float(formatted_exptime)
            exposure_times.append(float_exptime)

In [ ]:
##Gather ETC Versions
for number in results_list:
    objnumber = requests.get(number)
    soup = bs.BeautifulSoup(objnumber.text, 'html.parser')
    content = soup.find("div", {"id": "content"})
    ver = content.find(text=re.compile('ETC version')).strip().split('version')[-1].strip()
    etc_version.append(ver)

In [ ]:
df = pd.DataFrame({'Count Rate': count_rate, 'ExpTime': exposure_times, 'S/N': signal_to_noise, 'ETC Version': etc_version})
df